
# 🏪 Superstore Sales Analytics (Python, Pandas, Matplotlib)

**Objective:** Perform an end‑to‑end analysis of a Superstore sales dataset to discover revenue drivers, profit leaks, and actionable business recommendations.

**Dataset:** `Sample - Superstore.csv` (standard Kaggle/Excel sample). Please keep the file in the same folder as this notebook.

**Deliverables**
- Clean, reproducible notebook with clear explanations
- Business‑ready visuals
- Final insights & recommendations
- Optional HTML export for portfolio/GitHub



## ⚙️ Setup
Install/import libraries and set display options.


In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_colwidth', 120)
print('Versions -> pandas:', pd.__version__)



## 📥 Load the Dataset
We read the CSV with a safe encoding and parse dates later to keep control of types.


In [ ]:

# If running on Colab, mount drive or upload first. Locally, just keep the CSV next to the notebook.
CSV_PATH = "Sample - Superstore.csv"

# Read with fallback encoding
try:
    df = pd.read_csv(CSV_PATH, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(CSV_PATH, encoding="latin1")

print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
df.head(3)



## 🔎 Quick Inspection
Basic structure, column types, and missing values.


In [ ]:

print("Column dtypes:")
print(df.dtypes.sort_index())

missing = df.isna().sum().sort_values(ascending=False)
missing_pct = (missing/len(df))*100
overview = pd.DataFrame({"Missing Count": missing, "Missing %": missing_pct.round(2)})
overview[overview["Missing Count"]>0]



## 🧹 Data Cleaning & Preparation
- Convert date features to proper datetime
- Standardize column names (snake_case)
- Create helpful features: `order_year`, `order_month`, `profit_margin_pct`


In [ ]:

# Standardize column names
df.columns = (
    df.columns
      .str.strip()
      .str.replace(' ', '_')
      .str.replace('-', '_')
      .str.replace('/', '_')
      .str.lower()
)

# Parse date columns if present
for col in ["order_date", "ship_date"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Feature engineering
if {"profit","sales"}.issubset(df.columns):
    df["profit_margin_pct"] = np.where(df["sales"]!=0, (df["profit"]/df["sales"])*100, np.nan)

if "order_date" in df.columns:
    df["order_year"]  = df["order_date"].dt.year
    df["order_month"] = df["order_date"].dt.to_period("M").astype(str)

print("Prepared columns:", sorted(df.columns))
df.head(3)



## 📊 Descriptive Statistics
High‑level summary of numeric features.


In [ ]:

desc = df.describe(include='all', datetime_is_numeric=True).T
desc



## 📈 Exploratory Data Analysis
### 1) Total Sales & Profit by Category


In [ ]:

group_cols = []
if "category" in df.columns: group_cols.append("category")
elif "product_category" in df.columns: group_cols.append("product_category")

if group_cols and {"sales","profit"}.issubset(df.columns):
    cat = df.groupby(group_cols).agg(total_sales=("sales","sum"), total_profit=("profit","sum")).reset_index()
    display(cat.sort_values("total_sales", ascending=False))

    # Plot
    plt.figure(figsize=(8,5))
    x = cat[group_cols[0]].astype(str)
    y = cat["total_sales"]
    plt.bar(x, y)
    plt.title("Total Sales by Category")
    plt.xlabel(group_cols[0].title())
    plt.ylabel("Sales")
    plt.xticks(rotation=20, ha="right")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8,5))
    y2 = cat["total_profit"]
    plt.bar(x, y2)
    plt.title("Total Profit by Category")
    plt.xlabel(group_cols[0].title())
    plt.ylabel("Profit")
    plt.xticks(rotation=20, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("Category and sales/profit columns not found; skipping this section.")



### 2) Sales by Region & Segment (if available)


In [ ]:

if {"region","sales"}.issubset(df.columns):
    reg = df.groupby("region", as_index=False)["sales"].sum().sort_values("sales", ascending=False)
    display(reg)

    plt.figure(figsize=(8,5))
    plt.bar(reg["region"].astype(str), reg["sales"])
    plt.title("Total Sales by Region")
    plt.xlabel("Region")
    plt.ylabel("Sales")
    plt.xticks(rotation=20, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("Region column not found; skipping.")

if {"segment","profit"}.issubset(df.columns):
    seg = df.groupby("segment", as_index=False)["profit"].sum().sort_values("profit", ascending=False)
    display(seg)

    plt.figure(figsize=(8,5))
    plt.bar(seg["segment"].astype(str), seg["profit"])
    plt.title("Total Profit by Segment")
    plt.xlabel("Segment")
    plt.ylabel("Profit")
    plt.xticks(rotation=20, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("Segment column not found; skipping.")



### 3) Monthly Trend of Sales


In [ ]:

if "order_month" in df.columns and "sales" in df.columns:
    monthly = df.groupby("order_month", as_index=False)["sales"].sum()
    # Ensure chronological sort for YYYY-MM
    monthly = monthly.sort_values("order_month")
    display(monthly.head())

    plt.figure(figsize=(10,4))
    plt.plot(monthly["order_month"].astype(str), monthly["sales"], marker="o")
    plt.title("Monthly Sales Trend")
    plt.xlabel("Order Month")
    plt.ylabel("Sales")
    plt.xticks(rotation=60, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("order_month or sales missing; skipping.")



### 4) Top 10 Products & Customers by Sales


In [ ]:

if {"product_name","sales"}.issubset(df.columns):
    top_products = df.groupby("product_name", as_index=False)["sales"].sum().nlargest(10, "sales")
    display(top_products)

    plt.figure(figsize=(10,5))
    plt.barh(top_products["product_name"].astype(str), top_products["sales"])
    plt.gca().invert_yaxis()
    plt.title("Top 10 Products by Sales")
    plt.xlabel("Sales")
    plt.ylabel("Product")
    plt.tight_layout()
    plt.show()
else:
    print("product_name or sales missing; skipping products.")

if {"customer_name","sales"}.issubset(df.columns):
    top_customers = df.groupby("customer_name", as_index=False)["sales"].sum().nlargest(10, "sales")
    display(top_customers)

    plt.figure(figsize=(10,5))
    plt.barh(top_customers["customer_name"].astype(str), top_customers["sales"])
    plt.gca().invert_yaxis()
    plt.title("Top 10 Customers by Sales")
    plt.xlabel("Sales")
    plt.ylabel("Customer")
    plt.tight_layout()
    plt.show()
else:
    print("customer_name or sales missing; skipping customers.")



### 5) Discount vs Profit


In [ ]:

if {"discount","profit"}.issubset(df.columns):
    plt.figure(figsize=(6,5))
    plt.scatter(df["discount"], df["profit"], alpha=0.4)
    plt.title("Discount vs Profit")
    plt.xlabel("Discount")
    plt.ylabel("Profit")
    plt.tight_layout()
    plt.show()

    corr = df[["discount","profit"]].corr().iloc[0,1]
    print(f"Correlation between discount and profit: {corr:.3f}")
else:
    print("discount or profit missing; skipping.")



## 📑 Business Tables (Pivots)


In [ ]:

# Profitability by category and region (if available)
idx = [c for c in ["category","region"] if c in df.columns]
if idx and "profit_margin_pct" in df.columns:
    pivot_margin = pd.pivot_table(df, index=idx, values="profit_margin_pct", aggfunc="mean").sort_values("profit_margin_pct", ascending=False)
    display(pivot_margin.head(10))
else:
    print("Needed columns not present; skipping pivot.")



## 🧠 Insights & Recommendations
> Replace placeholders with your actual observations after reviewing the outputs above.

- **High performers:** Technology appears to lead sales; verify margins to ensure sustainable growth.
- **Profit leaks:** Furniture may have lower margins; review discounting strategy and shipping costs.
- **Geography:** Double‑down on high‑profit regions; improve service levels in low‑profit regions.
- **Customers:** Nurture top customers with loyalty offers; review low‑profit accounts.

**Recommended Actions**
1. Reduce excessive discounts where profit correlation is negative.
2. Prioritize profitable sub‑categories for campaigns.
3. Optimize shipping modes to cut costs and returns.



## 📤 Export Cleaned Data & Report (Optional)


In [ ]:

# Save cleaned dataset
clean_path = "superstore_clean.csv"
df.to_csv(clean_path, index=False)
print("Saved cleaned data ->", clean_path)

# Save key tables
outputs = {
    "category_sales_profit.csv": 'cat' in globals(),
    "region_sales.csv": 'reg' in globals(),
    "segment_profit.csv": 'seg' in globals(),
    "top_products.csv": 'top_products' in globals(),
    "top_customers.csv": 'top_customers' in globals()
}
for fname, exists in outputs.items():
    if exists:
        eval_name = fname.split('.')[0]
        obj = globals().get(eval_name)
        if obj is not None:
            obj.to_csv(fname, index=False)
            print("Saved ->", fname)

# Export to HTML (static) for portfolio
try:
    import nbconvert
    print("You can export via: File > Save and Export Notebook As > HTML")
except Exception as e:
    print("nbconvert not available; export via Jupyter UI.")



---
**Author:** Your Name  

**Last updated:** 2025-11-11 10:32:55
